In [ ]:
%cd /ibex/user/slimhy/PADS/code
"""
Extracting features into HDF5 files for each split.
"""
import argparse
import torch
import numpy as np
import trimesh
import os
import zipfile
import json
from datasets.CoMPaT.compat3D import SegmentedMeshLoader
from datasets.metadata import class_to_idx


ZIP_PATH = "/home/slimhy/Documents/datasets/0_shapes/3DCoMPaT/zip/3DCoMPaT_ZIP.zip"
META_DIR = "/home/slimhy/Documents/3DCoMPaT/repo/metadata"

# Instantiating segment dataset
seg_dataset = SegmentedMeshLoader(
    filter_class=[class_to_idx("chair")],
    zip_path=ZIP_PATH,
    meta_dir=META_DIR,
    split="train",
    shuffle=True,
    get_instances=True,
    seed=0,
)

mesh_map = seg_dataset[1]

In [2]:
from util.misc import get_bb_vecs

PART_ID = 0

part_mesh = list(mesh_map.values())[PART_ID]
if isinstance(part_mesh, list):
    part_mesh = part_mesh[0]
bb_prim = part_mesh.bounding_box_oriented
centroid, vecs = get_bb_vecs(bb_prim)

In [ ]:
import k3d

# plot all points in bbox
plot = k3d.plot()

# plot centroid
plot += k3d.points(centroid, point_size=0.02, color=0x00ff00)

# add the mesh
plot += k3d.mesh(part_mesh.vertices, part_mesh.faces, color=0xefefef, opacity=0.8)

# plot all points
for idx, points in enumerate(bb_prim.vertices):
    plot += k3d.points(points, point_size=0.02, color=0xff0000, opacity=0.8)
    
# add bb as a besh
plot += k3d.mesh(bb_prim.vertices, bb_prim.faces, color=0xff00ff, opacity=0.2)
    
# plot u, v, w as lines
for v in vecs:
    plot += k3d.line([centroid, centroid + v], color=0x0000ff, opacity=0.8)
    
for v in vecs:
    plot += k3d.line([centroid, centroid - v], color=0x00ff00, opacity=0.8)
    
plot.display()

In [ ]:
import k3d

# plot all points in bbox
plot = k3d.plot()

def add_mesh(mesh, plot):
    plot += k3d.mesh(mesh.vertices, mesh.faces, color=0xefefef, opacity=0.8)
    return plot

# now plot all parts
part_mesh = list(mesh_map.values())
for mesh in part_mesh:
    if isinstance(mesh, list):
        for submesh in mesh:
            plot = add_mesh(submesh, plot)
    else:
        add_mesh(mesh, plot)

plot.display()